In [1]:
import pandas as pd

In [2]:
import dask
import dask.dataframe as dd
from dask import delayed
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/scaling/dask_em')


In [3]:
from dask_em.sampler.downsample.downsample import downsample_dd


In [4]:
path = '../datasets/'
A = dd.read_csv(path+'sample_msd.csv')
B = dd.read_csv(path+'sample_msd.csv')
A.set_index('id')
B.set_index('id')

,title,release,artist_name,duration,artist_familiarity,artist_hotttnesss,year
npartitions=1,,,,,,,
2,object,object,object,float64,float64,float64,int64
1000000,...,...,...,...,...,...,...


In [13]:
import string

import pandas as pd
from dask import threaded, delayed
from dask_em.sampler.downsample.dsprober import DownSampleProber
from dask import dataframe as dd

from dask_em.utils.cy_utils.stringcontainer import StringContainer
from dask_em.utils.py_utils.utils import get_str_cols, str2bytes, sample, split_df, \
    tokenize_strings_wsp, build_inv_index

# helper functions
def preprocess_table(dataframe, idcol):
    strcols = list(get_str_cols(dataframe))
    strcols.append(idcol)
    projdf = dataframe[strcols]
    objsc = StringContainer()
    for row in projdf.itertuples():
        colvalues = row[1:-1]
        uid = row[-1]
        strings = [colvalue.strip() for colvalue in colvalues if not pd.isnull(colvalue)]
        concat_row = str2bytes(' '.join(strings).lower())
        concat_row = concat_row.translate(None, string.punctuation)
        objsc.push_back(uid, concat_row)
    return objsc

def probe(objtc, objinvindex, yparam):
    objprobe = DownSampleProber()
    objprobe.probe(objtc, objinvindex, yparam)
    return objprobe

def lpostprocess(result_list):
    lids = set()
    for i in range(len(result_list)):
        result = result_list[i]
        lids.update(result.get_lids())
    lids = sorted(lids)
    return lids


def rpostprocess(result_list):
    rids = set()
    for i in range(len(result_list)):
        result = result_list[i]
        rids.update(result.get_rids())
    rids = sorted(rids)
    return rids


def select(table, ids):
    return table.loc[ids]

def get_meta(table):
    vals = []
    for col in table.columns:
        if table[col].dtype == 'int64':
            vals.append(0)
        elif table[col].dtype == 'float64':
            vals.append(0.0)
        elif table[col].dtype == 'bool':
            vals.append(False)
        elif table[col].dtype == 'object':
            vals.append('')
        else:
            raise ValueError(col)
    result = [tuple(vals)]
    meta = pd.DataFrame(result, columns=list(table.columns))
    return meta

In [16]:
lstopwords = ['the', 'of', 'in', 'and', 'version', 'and', 'to', 'you', 'i', 'love', 'me', 'album', 'live', 'my', 'on', 'for', 'de', 'it', 'best', 'is', 'la', 'vol', 'from', 'no', 'your', 'all']

rstopwords = lstopwords
stopwords = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its', 'on', 'that', 'the', 'to', 'of', 'was', 'were', 'will', 'with']

# stopwords


lstopwords = lstopwords + stopwords
rstopwords = rstopwords + stopwords

fraction = 0.01
y = 1

In [17]:
    ltokens = []
    for i in range(ltable.npartitions):
        lcat_strings = (delayed)(preprocess_table)(ltable.get_partition(i), lid)
        tokens = (delayed)(tokenize_strings_wsp)(lcat_strings, lstopwords)
        ltokens.append(tokens)

    invindex = (delayed)(build_inv_index)(ltokens)
    rsample = rtable.sample(fraction, random_state=0)
    rsample.set_index(rid)

    probe_rslts = []
    for i in range(rsample.npartitions):
        rcat_strings = (delayed)(preprocess_table)(rsample.get_partition(i), rid)
        rtokens = (delayed)(tokenize_strings_wsp)(rcat_strings, rstopwords)
        probe_rslt = (delayed)(probe)(rtokens, invindex, y)
        probe_rslts.append(probe_rslt)


[########################################] | 100% Completed |  0.5s


In [18]:
rresult = (delayed)(rpostprocess)(probe_rslts)

In [19]:
tt = rresult.compute()

[########################################] | 100% Completed |  2.9s


In [21]:
len(tt)

1000

In [23]:
tt = (delayed)(select)(rtable, rresult)

In [24]:
tt1 = tt.compute()

[########################################] | 100% Completed |  3.1s


In [25]:
tt1.head()

,id,title,release,artist_name,duration,artist_familiarity,artist_hotttnesss,year
35,900739.0,Good Things,The Real Rock,Shinehead,244.03546,0.529103,0.411364,0.0
127,196518.0,Choo Choo,Euro Party,London MC,247.90159,0.426840,0.334134,2008.0
237,351568.0,Duh Duh Da (Next Generation Mix),The Next Generation,Lidell Townsell,185.86077,0.431622,0.307084,0.0
560,238300.0,3 Mad P's,3 Mad P's/Experience/In Motion,Headhunter,421.14567,0.607734,0.448453,0.0
718,493367.0,We Gotta Get Together,80.81,The Right Ons,293.95546,0.558117,0.474894,2007.0


In [5]:
lstopwords = ['the', 'of', 'in', 'and', 'version', 'and', 'to', 'you', 'i', 'love', 'me', 'album', 'live', 'my', 'on', 'for', 'de', 'it', 'best', 'is', 'la', 'vol', 'from', 'no', 'your', 'all']

rstopwords = lstopwords
stopwords = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its', 'on', 'that', 'the', 'to', 'of', 'was', 'were', 'will', 'with']

# stopwords


lstopwords = lstopwords + stopwords
rstopwords = rstopwords + stopwords

A1, B1 = downsample_dd(A, B, 'id', 'id', 0.1, 1,lstopwords=lstopwords, rstopwords=rstopwords)

In [6]:
from dask.diagnostics import *

In [7]:
pbar = ProgressBar()
pbar.register()

In [10]:
from distributed import Client

In [14]:
from distributed.diagnostics import progress

In [11]:
client = Client()

In [17]:
B1.persist()

,id,title,release,artist_name,duration,artist_familiarity,artist_hotttnesss,year
npartitions=1,,,,,,,,
None,int64,object,object,object,float64,float64,float64,int64
None,...,...,...,...,...,...,...,...


In [18]:
C1 = B1.compute()

In [19]:
B1.head()

,id,title,release,artist_name,duration,artist_familiarity,artist_hotttnesss,year
2,681317.0,Flying in the sky (feat. Feat. Ange Vocal Mix),Flying In the Sky (Original mix),Channel One,432.95302,0.531467,0.324757,0.0
35,900739.0,Good Things,The Real Rock,Shinehead,244.03546,0.529103,0.411364,0.0
127,196518.0,Choo Choo,Euro Party,London MC,247.90159,0.426840,0.334134,2008.0
237,351568.0,Duh Duh Da (Next Generation Mix),The Next Generation,Lidell Townsell,185.86077,0.431622,0.307084,0.0
291,851695.0,Martin,Call To Arms,Sick Of It All,166.86975,0.690626,0.420802,1999.0
